In [13]:
import numpy as np
import pandas as pd
import math
from scipy.sparse import csr_matrix
import collections

# import data
trainData = np.loadtxt(fname="./datasets/trainData.txt",dtype=int)
testData = np.loadtxt(fname="./datasets/testData.txt",dtype=int)
trainLabel = np.loadtxt(fname="./datasets/trainLabel.txt",dtype=int)
testLabel = np.loadtxt(fname="./datasets/testLabel.txt",dtype=int)
word  = np.loadtxt(fname="./datasets/words.txt",dtype=str)
words = pd.read_csv("./datasets/words.txt", names = ['word'])


# Part A

In [14]:
trainData=csr_matrix((np.ones(np.size(trainData,0)), (trainData[:,0]-1, trainData[:,1]-1)),\
                           shape=(max(trainData[:,0]), 6967)).todense()
testData=csr_matrix((np.ones(np.size(testData,0)), (testData[:,0]-1, testData[:,1]-1)),\
                           shape=(max(testData[:,0]), 6967)).todense()


In [15]:
trainData = np.array(trainData)
testData = np.array(testData)
trainLabel=trainLabel.reshape(1500,1)
testLabel=testLabel.reshape(1500,1)

In [16]:
# [features, label]
train = np.hstack([trainData,trainLabel])
test = np.hstack([testData,testLabel])
print(trainData.shape,type(trainData))
print(trainLabel.shape,type(trainLabel))
print(testData.shape,type(testData))
print(testLabel.shape,type(testLabel))


(1500, 6967) <class 'numpy.ndarray'>
(1500, 1) <class 'numpy.ndarray'>
(1500, 6967) <class 'numpy.ndarray'>
(1500, 1) <class 'numpy.ndarray'>


In [17]:
print(train)
print(test)
print(train.shape,type(train))
print(test.shape,type(test))

[[0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 ...
 [0. 0. 0. ... 0. 0. 2.]
 [0. 0. 0. ... 0. 0. 2.]
 [0. 0. 0. ... 0. 0. 2.]]
[[0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 ...
 [0. 0. 0. ... 0. 0. 2.]
 [0. 0. 0. ... 0. 0. 2.]
 [0. 0. 0. ... 0. 0. 2.]]
(1500, 6968) <class 'numpy.ndarray'>
(1500, 6968) <class 'numpy.ndarray'>


In [18]:
cat1 = train[train[:,-1]==1]
cat2 = train[train[:,-1]==2]
cat2[:,-1]=1

cat1_sum = np.sum(cat1,axis=0)
cat2_sum = np.sum(cat2,axis=0)

cat1_sum[:-1] +=1
cat2_sum[:-1] +=1

cat1_sum[-1] +=2
cat2_sum[-1] +=2

cat1_prob = cat1_sum/ cat1_sum[-1]
cat2_prob = cat2_sum/ cat2_sum[-1]
cat1_log = np.log(cat1_prob)
cat2_log = np.log(cat2_prob)

diff = np.abs(cat1_log - cat2_log)
# remove the tag
diff_no_label = np.abs(diff[:-1])
index = np.argsort(diff_no_label)[::-1]
top_10_index = index[:10]
w = words['word'].to_list()
for i in top_10_index:
    print('word_index: %6d\tword: %10s \tlog_diff: '%(i,w[i]) + str(diff[i]))

word_index:    192	word:  christian 	log_diff: 3.5835189384561104
word_index:   5239	word:   religion 	log_diff: 3.5115454388310208
word_index:   4661	word:    atheism 	log_diff: 3.2958368660043296
word_index:   2436	word:      books 	log_diff: 3.242592351485517
word_index:   4828	word:    library 	log_diff: 3.218875824868201
word_index:   1238	word: christians 	log_diff: 3.218875824868201
word_index:    198	word:  religious 	log_diff: 3.091042453358316
word_index:   3162	word:  libraries 	log_diff: 3.091042453358316
word_index:   6897	word:      novel 	log_diff: 3.091042453358316
word_index:   6526	word: characters 	log_diff: 2.9957322735539913


# Part B

In [19]:
N = 1500
theta_c1 = 0.5
theta_c2 = 0.5
theta_a1 = train[train[:,-1]==1][:,:-1]
theta_a2 = train[train[:,-1]==2][:,:-1]
theta_a1 = np.sum(theta_a1,axis=0)+1
theta_a2 = np.sum(theta_a2,axis=0)+1
theta_a1 = theta_a1/752
theta_a2 = theta_a2/752

In [20]:
prob_1 = trainData*np.log(theta_a1) + (1-trainData)*np.log(1-theta_a1)
prob_2 = trainData*np.log(theta_a2) + (1-trainData)*np.log(1-theta_a2)
prob_1 = np.sum(prob_1,axis=1)
prob_2 = np.sum(prob_2,axis=1)
print(prob_1.shape)

(1500,)


In [21]:

cat = prob_1/(prob_1+prob_2)
cat[cat > 0.5] = 2
cat[cat <= 0.5] = 1
cat.reshape((1500,1))
cat = np.array(list(cat))
tL = np.array(list(trainLabel.flatten()))
print('Training Accuracy')
print(np.count_nonzero(cat == tL)/1500)

Training Accuracy
0.9026666666666666


In [22]:
# N = 1500
# theta_c1 = 0.5
# theta_c2 = 0.5
# theta_a1 = train[train[:,-1]==1][:,:-1]
# theta_a2 = train[train[:,-1]==2][:,:-1]
# theta_a1 = np.sum(theta_a1,axis=0)+1
# theta_a2 = np.sum(theta_a2,axis=0)+1
# theta_a1 = theta_a1/752
# theta_a2 = theta_a2/752

In [23]:
prob_1 = testData*np.log(theta_a1) + (1-testData)*np.log(1-theta_a1)
prob_2 = testData*np.log(theta_a2) + (1-testData)*np.log(1-theta_a2)
prob_1 = np.sum(prob_1,axis=1)
prob_2 = np.sum(prob_2,axis=1)

In [24]:
cat = prob_1/(prob_1+prob_2)
cat[cat > 0.5] = 2
cat[cat <= 0.5] = 1
cat.reshape((1500,1))
cat = np.array(list(cat))
tL = np.array(list(testLabel.flatten()))
print('Testing Accuracy')
print(np.count_nonzero(cat == tL)/1500)

Testing Accuracy
0.7446666666666667
